### Install dependencies

In [82]:
%env HDF5_USE_FILE_LOCKING=FALSE

env: HDF5_USE_FILE_LOCKING=FALSE


In [83]:
import bpnet
from bpnet.cli.contrib import ContribFile
from bpnet.plot.tracks import plot_tracks, to_neg

import os
import uuid
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import clear_output, HTML
from pathlib import Path
import pandas as pd
import numpy as np
clear_output()

#### Optional: Setup wandb

In [84]:
import wandb

wandb.init(project='bpnet-training', entity='an1lam')

2020-09-02 16:18:47,859 [INFO] system metrics and metadata threads started
2020-09-02 16:18:47,860 [INFO] checking resume status, waiting at most 10 seconds
2020-09-02 16:18:47,942 [INFO] resuming run from id: UnVuOnYxOjJ4d2M5aDdpOmJwbmV0LXRyYWluaW5nOmFuMWxhbQ==
2020-09-02 16:18:47,948 [INFO] upserting run before process can begin, waiting at most 10 seconds
2020-09-02 16:18:48,058 [INFO] saving pip packages
2020-09-02 16:18:48,060 [INFO] initializing streaming files api
2020-09-02 16:18:48,061 [INFO] unblocking file change observer, beginning sync with W&B servers


W&B Run: https://app.wandb.ai/an1lam/bpnet-training/runs/2xwc9h7i

2020-09-02 16:18:48,066 [INFO] shutting down system stats and metadata service
2020-09-02 16:18:48,784 [INFO] file/dir modified: /home/stephenmalina/project/src/wandb/run-20200902_161847-2xwc9h7i/config.yaml
2020-09-02 16:18:48,861 [INFO] stopping streaming files and file change observer
2020-09-02 16:18:48,873 [INFO] file/dir created: /home/stephenmalina/project/src/wandb/run-20200902_161847-2xwc9h7i/wandb-summary.json
2020-09-02 16:18:48,875 [INFO] file/dir created: /home/stephenmalina/project/src/wandb/run-20200902_161847-2xwc9h7i/wandb-metadata.json
2020-09-02 16:18:48,880 [INFO] file/dir created: /home/stephenmalina/project/src/wandb/run-20200902_161847-2xwc9h7i/wandb-history.jsonl
2020-09-02 16:18:48,882 [INFO] file/dir created: /home/stephenmalina/project/src/wandb/run-20200902_161847-2xwc9h7i/requirements.txt
2020-09-02 16:18:48,884 [INFO] file/dir created: /home/stephenmalina/project/src/wandb/run-20200902_161847-2xwc9h7i/wandb-events.jsonl
2020-09-02 16:18:48,891 [INFO] file/

In [85]:
# config variables
n_reps = 5

# file paths
config_dir = Path('./bpnet/') 

model_config_fname = 'ChIP-nexus-default.gin'
data_config_fname = 'ChIP-nexus.dataspec.yml'

timestamp = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
output_dir = f'/home/stephenmalina/project/dat/res-bpnet-training-{timestamp}'
output_dir

'/home/stephenmalina/project/dat/res-bpnet-training-2020-09-02-16-18-50'

In [86]:
os.makedirs(output_dir, exist_ok=True)
os.makedirs(os.path.join(output_dir, 'output_ensemble'), exist_ok=True)

In [87]:
!cat {config_dir}/{data_config_fname}

task_specs:
  Oct4:
    tracks:
    - /home/stephenmalina/project/dat/bpnet-manuscript-data/data/chip-nexus/Oct4/counts.pos.bw
    - /home/stephenmalina/project/dat/bpnet-manuscript-data/data/chip-nexus/Oct4/counts.neg.bw
    peaks: /home/stephenmalina/project/dat/bpnet-manuscript-data/data/chip-nexus/Oct4/idr-optimal-set.summit.bed.gz
  Sox2:
    tracks:
    - /home/stephenmalina/project/dat/bpnet-manuscript-data/data/chip-nexus/Sox2/counts.pos.bw
    - /home/stephenmalina/project/dat/bpnet-manuscript-data/data/chip-nexus/Sox2/counts.neg.bw
    peaks: /home/stephenmalina/project/dat/bpnet-manuscript-data/data/chip-nexus/Sox2/idr-optimal-set.summit.bed.gz
  Nanog:
    tracks:
    - /home/stephenmalina/project/dat/bpnet-manuscript-data/data/chip-nexus/Nanog/counts.pos.bw
    - /home/stephenmalina/project/dat/bpnet-manuscript-data/data/chip-nexus/Nanog/counts.neg.bw
    peaks: /home/stephenmalina/project/dat/bpnet-manuscript-data/data/chip-nexus/Nanog/idr-optimal-set.summit.bed.gz
  Klf4

### Data stats

In [88]:
# chromsomome names of differnet peaks
!zcat /home/stephenmalina/project/dat/bpnet-manuscript-data/data/chip-nexus/Sox2/idr-optimal-set.summit.bed.gz \
    | cut -f 1 | sort -u

chr1
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr2
chr3
chr4
chr5
chr6
chr7
chr8
chr9
chrX
chrY


Each task (or TF) can specify a set of peaks associated with it. Here are the number of peaks per TF we will use in this tutorial:

In [89]:
tasks = ['Oct4', 'Sox2', 'Nanog', 'Klf4']

# number of peaks per task
for task in tasks:
    print(task)
    data_dir = '/home/stephenmalina/project/dat/bpnet-manuscript-data'
    !zcat {data_dir}/data/chip-nexus/{task}/idr-optimal-set.summit.bed.gz | wc -l

Oct4
25849
Sox2
10999
Nanog
56459
Klf4
57601


## 2. Train the model

Having specified `dataspec.yml`, we are now ready to train the model with 

```
bpnet train <dataspec.yml> <output dir> [optional flags]`
```


We will use a pre-made model [bpnet9](../bpnet/premade/bpnet9.gin) as a starting point and modify a few parameters specified in the config.gin file. Specifically, we will 
- train the model only on chromosomes 16-19
- evaluate the model on chromosome 2
- use only 3 layers of dilated convolutions 
- use an input sequence length of 200 bp and accordingly lower the augmentation shift to 100 bp

In [90]:
!cat {config_dir}/{model_config_fname} 
# NOTE: test_chr will be also excluded similar to 'exclude_chr'

b_loss_weight = 0
c_loss_weight = 10
p_loss_weight = 1
filters = 64
tconv_kernel_size = 25
lr = 0.004
n_dil_layers = 9
train.batch_size = 128
merge_profile_reg = False
dataspec = 'ChIP-nexus.dataspec.yml'

batchnorm = False

padding = 'same'
seq_width = 1000

tasks = ['Oct4', 'Sox2', 'Nanog', 'Klf4']


Have a look at the original gin file of bpnet9 here: https://github.com/kundajelab/bpnet/blob/master/bpnet/premade/bpnet9-ginspec.gin. For more information on using gin files see <https://github.com/google/gin-config>. 

To track model training and evaluation, we will use [wandb](http://wandb.com/) by adding `--wandb=avsec/bpnet-demo` to `bpnet train`. You can navigate to https://app.wandb.ai/avsec/bpnet-demo to see the training progress.

Let's train!

In [91]:
# setup all the file paths
example_model_dir = os.path.join(output_dir, 'output_ensemble', '0')

In [92]:
# Train for at most 10 epochs
for i in range(n_reps):
    # setup a new run_id (could be done automatically, but then the output directory would change)
    run_id = datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + "_" + str(uuid.uuid4())
    !cd {config_dir} && bpnet train {data_config_fname} --premade=bpnet9 \
        --config={model_config_fname} {output_dir} \
        --run-id '{run_id}' --wandb=an1lam/bpnet-training \
        --override='train.epochs=10; train.seed={i}'
    # softlink the new output directory
    !rm -rf {output_dir}/output_ensemble/{i} && ln -srf {output_dir}/{run_id} {output_dir}/output_ensemble/{i}

Using TensorFlow backend.
2020-09-02 16:18:52.941582: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0

2020-09-02 16:18:54,352 [WARNING] From /opt/conda/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

2020-09-02 16:18:54,763 [INFO] NumExpr defaulting to 8 threads.
/opt/conda/lib/python3.7/site-packages/bpnet/plot/heatmaps.py:6: MatplotlibDeprecationWarning: 
The mpl_toolkits.axes_grid1.colorbar module was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use matplotlib.colorbar instead.
  from mpl_toolkits.axes_grid1.colorbar import colorbar
INFO [09-02 16:18:56] Using wandb. Running wandb.init()
wandb: Tracking run with wandb version 0.9.6
wandb: Run data is saved locally in ../../dat/res-bpnet-training-2020-09-02-16-18-50/run-20200902_161856-2020-09-02_16-18-51_d4fb7dfe-59f4-4bfb-9f90-b418a3666

Epoch 1/10
2020-09-02 16:19:10.195726: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-09-02 16:19:10.423217: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
710/710 [==============================] - 205s 288ms/step - loss: 3566.4380 - Oct4/profile_loss: 852.2895 - Oct4/counts_loss: 0.5176 - Sox2/profile_loss: 486.7343 - Sox2/counts_loss: 0.8424 - Nanog/profile_loss: 1251.1001 - Nanog/counts_loss: 1.2050 - Klf4/profile_loss: 939.1108 - Klf4/counts_loss: 1.1553 - val_loss: 3385.2158 - val_Oct4/profile_loss: 837.8087 - val_Oct4/counts_loss: 0.4279 - val_Sox2/profile_loss: 467.3936 - val_Sox2/counts_loss: 0.5100 - val_Nanog/profile_loss: 1134.5624 - val_Nanog/counts_loss: 0.9386 - val_Klf4/profile_loss: 919.9731 - val_Klf4/counts_loss: 0.6713anog/profile_loss: 1582.5187 - Nanog/counts_loss: 1.9811 - Klf4/profile_loss: 979.1657 - Klf4/counts_l - ETA:

710/710 [==============================] - 197s 277ms/step - loss: 3154.2753 - Oct4/profile_loss: 801.6806 - Oct4/counts_loss: 0.3026 - Sox2/profile_loss: 449.8455 - Sox2/counts_loss: 0.2201 - Nanog/profile_loss: 1002.7979 - Nanog/counts_loss: 0.6296 - Klf4/profile_loss: 883.9320 - Klf4/counts_loss: 0.4497 - val_loss: 3204.6804 - val_Oct4/profile_loss: 814.5950 - val_Oct4/counts_loss: 0.2854 - val_Sox2/profile_loss: 457.9260 - val_Sox2/counts_loss: 0.2165 - val_Nanog/profile_loss: 1032.2162 - val_Nanog/counts_loss: 0.6128 - val_Klf4/profile_loss: 884.3953 - val_Klf4/counts_loss: 0.4401ss: 806.7486 - Oct4/counts_loss: 0.3106 - Sox2/profile_loss: 452.9197 - Sox2/counts_loss: 0.2334 - Nanog/profile_loss:  - ETA: 2:03 - loss: 3157.9759 - Oct4/profile_loss: 801.5841 - Oct4/counts_loss: 0.3028 - Sox2/profile_loss: 450.8175 - Sox2/counts_loss: 0.2254 - Nanog/pro - ETA: 48s - loss: 3152.0762 - Oct4/profile_loss: 800.6272 - Oct4/counts_loss: 0.3036 - Sox2/profile_loss: 449.6579 - Sox2/counts_lo

INFO [09-02 17:01:02] Running the evaluation report
Executing:  10%|███▏                           | 3/29 [00:03<00:33,  1.28s/cell]2020-09-02 17:01:06.817030: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2020-09-02 17:01:06.817825: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55dfaa22cc30 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-09-02 17:01:06.817857: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-09-02 17:01:06.820802: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-09-02 17:01:06.888152: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-02 17:01:06.888773: I tensorflow/compiler/xla/service/service.

wandb: Waiting for W&B process to finish, PID 4325
 'Sox2/profile/binsize=10/imbalance': 0.05612442985432679, 'Sox2/counts/mse': 0.25067776, 'Sox2/counts/var_explained': 0.3330099582672119, 'Sox2/counts/pearsonr': 0.5771318501153297, 'Sox2/counts/spearmanr': 0.5396412516368724, 'Sox2/counts/mad': 0.40291572, 'Nanog/profile/binsize=1/auprc': 0.45204433154678675, 'Nanog/profile/binsize=1/random_auprc': 0.005396128595434531, 'Nanog/profile/binsize=1/n_positives': 411441, 'Nanog/profile/binsize=1/frac_ambigous': 0.058152092850969256, 'Nanog/profile/binsize=1/imbalance': 0.005393806853993239, 'Nanog/profile/binsize=10/auprc': 0.7266661488124783, 'Nanog/profile/binsize=10/random_auprc': 0.040264529325654864, 'Nanog/profile/binsize=10/n_positives': 247668, 'Nanog/profile/binsize=10/frac_ambigous': 0.24060291393999259, 'Nanog/profile/binsize=10/imbalance': 0.04026888195270616, 'Nanog/counts/mse': 0.67656076, 'Nanog/counts/var_explained': 0.35717469453811646, 'Nanog/counts/pearsonr': 0.59867331

...

710/710 [==============================] - 198s 279ms/step - loss: 3140.4525 - Oct4/profile_loss: 800.9910 - Oct4/counts_loss: 0.2896 - Sox2/profile_loss: 449.4956 - Sox2/counts_loss: 0.2122 - Nanog/profile_loss: 993.6357 - Nanog/counts_loss: 0.6022 - Klf4/profile_loss: 881.1188 - Klf4/counts_loss: 0.4171 - val_loss: 3193.7645 - val_Oct4/profile_loss: 809.7529 - val_Oct4/counts_loss: 0.4020 - val_Sox2/profile_loss: 458.0699 - val_Sox2/counts_loss: 0.2888 - val_Nanog/profile_loss: 1024.9146 - val_Nanog/counts_loss: 0.6134 - val_Klf4/profile_loss: 883.1248 - val_Klf4/counts_loss: 0.4860profile_loss: 877.8514 -  - ETA: 2:05 - loss: 3136.1471 - Oct4/profile_loss: 800.7938 - Oct4/counts_loss: 0.2920 - Sox2/profile_loss: 449.0603 - Sox2/counts_loss: 0.2136 - Nanog/p - ETA: 1:20 - loss: 3139.9713 - Oct4/profile_loss: 801.0407 - Oct4/counts_loss: 0.2902 - Sox2/profile_loss: 449.1120 - Sox2/counts_loss: 0.2114 - Nanog/profile_loss: 994.4421 - Nanog/counts_loss: 0.6023 - Klf4/profile_loss: 880.1

INFO [09-02 19:57:44] Running the evaluation report
Executing:  10%|███▏                           | 3/29 [00:03<00:33,  1.28s/cell]2020-09-02 19:57:48.725111: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2020-09-02 19:57:48.726062: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55fe5e101e10 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-09-02 19:57:48.726110: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-09-02 19:57:48.730504: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-09-02 19:57:48.800165: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-02 19:57:48.800696: I tensorflow/compiler/xla/service/service.

wandb: Waiting for W&B process to finish, PID 6800
ox2/profile/binsize=10/imbalance': 0.05608734904701838, 'Sox2/counts/mse': 0.29732144, 'Sox2/counts/var_explained': 0.3124459981918335, 'Sox2/counts/pearsonr': 0.5707770475987016, 'Sox2/counts/spearmanr': 0.5496020381015704, 'Sox2/counts/mad': 0.39270914, 'Nanog/profile/binsize=1/auprc': 0.4520624791149324, 'Nanog/profile/binsize=1/random_auprc': 0.005521350952017978, 'Nanog/profile/binsize=1/n_positives': 411913, 'Nanog/profile/binsize=1/frac_ambigous': 0.05827248259173293, 'Nanog/profile/binsize=1/imbalance': 0.005400284822930656, 'Nanog/profile/binsize=10/auprc': 0.7297290655162583, 'Nanog/profile/binsize=10/random_auprc': 0.0413535346029784, 'Nanog/profile/binsize=10/n_positives': 248015, 'Nanog/profile/binsize=10/frac_ambigous': 0.24117623092498394, 'Nanog/profile/binsize=10/imbalance': 0.040352779105162905, 'Nanog/counts/mse': 0.6081712, 'Nanog/counts/var_explained': 0.33881711959838867, 'Nanog/counts/pearsonr': 0.609545833952243

wandb:                          eval/valid-peaks/Sox2/counts/mse 0.2900170683860779
wandb:                eval/valid-peaks/Sox2/counts/var_explained 0.326099157333374
wandb:                     eval/valid-peaks/Sox2/counts/pearsonr 0.5849559355397783
wandb:                    eval/valid-peaks/Sox2/counts/spearmanr 0.552382947580848
wandb:                          eval/valid-peaks/Sox2/counts/mad 0.3888988196849823
wandb:            eval/valid-peaks/Nanog/profile/binsize=1/auprc 0.42267641053344684
wandb:     eval/valid-peaks/Nanog/profile/binsize=1/random_auprc 0.005264769703912481
wandb:      eval/valid-peaks/Nanog/profile/binsize=1/n_positives 131131
wandb:    eval/valid-peaks/Nanog/profile/binsize=1/frac_ambigous 0.05827785356980446
wandb:        eval/valid-peaks/Nanog/profile/binsize=1/imbalance 0.005276866583254742
wandb:           eval/valid-peaks/Nanog/profile/binsize=10/auprc 0.7049438589953516
wandb:    eval/valid-peaks/Nanog/profile/binsize=10/random_auprc 0.0395907644512281


wandb:         eval/train-peaks/Klf4/profile/binsize=1/imbalance 0.0026058547813472355
wandb:            eval/train-peaks/Klf4/profile/binsize=10/auprc 0.5114899381713155
wandb:     eval/train-peaks/Klf4/profile/binsize=10/random_auprc 0.02834283894173522
wandb:      eval/train-peaks/Klf4/profile/binsize=10/n_positives 149686
wandb:    eval/train-peaks/Klf4/profile/binsize=10/frac_ambigous 0.31823936319015284
wandb:        eval/train-peaks/Klf4/profile/binsize=10/imbalance 0.02791866905952847
wandb:                          eval/train-peaks/Klf4/counts/mse 0.4974331855773926
wandb:                eval/train-peaks/Klf4/counts/var_explained 0.38057005405426025
wandb:                     eval/train-peaks/Klf4/counts/pearsonr 0.6181013182560472
wandb:                    eval/train-peaks/Klf4/counts/spearmanr 0.6207310209427741
wandb:                          eval/train-peaks/Klf4/counts/mad 0.5355029106140137
wandb:              eval/train-peaks/avg/profile/binsize=1/auprc 0.30687015480873

In [93]:
! echo {output_dir}
! echo
! ls {output_dir}/output_ensemble/0

/home/stephenmalina/project/dat/res-bpnet-training-2020-09-02-16-18-50

bpnet-train.kwargs.json			    history.csv
config.gin				    input-config.gin
config.gin.json				    log
dataspec.yml				    model.h5
evaluate.html				    note_params.json
evaluate.ipynb				    seq_model.pkl
evaluation.valid.json			    wandb.json
events.out.tfevents.1599063546.gregor-3-vm


In [94]:
!ls -latr {example_model_dir}/

total 14976
-rw-r--r--  1 stephenmalina stephenmalina       2 Sep  2 16:18 note_params.json
drwxr-xr-x  2 stephenmalina stephenmalina    4096 Sep  2 16:18 log
-rw-r--r--  1 stephenmalina stephenmalina     328 Sep  2 16:18 wandb.json
-rw-r--r--  1 stephenmalina stephenmalina     302 Sep  2 16:18 input-config.gin
-rw-r--r--  1 stephenmalina stephenmalina     579 Sep  2 16:18 bpnet-train.kwargs.json
-rw-r--r--  1 stephenmalina stephenmalina    1562 Sep  2 16:18 dataspec.yml
-rw-r--r--  1 stephenmalina stephenmalina    5943 Sep  2 16:18 config.gin
-rw-r--r--  1 stephenmalina stephenmalina    4191 Sep  2 16:18 config.gin.json
-rw-r--r--  1 stephenmalina stephenmalina    3743 Sep  2 16:51 history.csv
-rw-r--r--  1 stephenmalina stephenmalina 1733204 Sep  2 16:51 events.out.tfevents.1599063546.gregor-3-vm
-rw-r--r--  1 stephenmalina stephenmalina 1810128 Sep  2 16:52 model.h5
-rw-r--r--  1 stephenmalina stephenmalina 2401402 Sep  2 16:52 seq_model.pkl
-rw-r--r--  1 stephenmalina stephenmalina